In [ ]:
import tensorflow as tf
from tensorflow.keras import  layers, models, datasets
import numpy as np
import PIL
import PIL.Image
import pathlib, os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_dir = "drive/MyDrive/nature_12K/inaturalist_12K/train"
data_dir = pathlib.Path(data_dir)


In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
def model_creation(cnn_params, ff_param, input_shape, nr_classes,data_aug,dropout,batch_norm):

  model = models.Sequential()
  data_augmentation = tf.keras.Sequential([layers.RandomFlip("horizontal_and_vertical"),layers.RandomRotation(0.2),])
  if(data_aug):
    model.add(data_augmentation)
  for param in cnn_params:
      nr_filter = param[0]
      size_filter = param[1]
      filter_activation = param[2]
      model.add(layers.Conv2D(nr_filter, (size_filter,size_filter), activation= filter_activation, input_shape= input_shape))
      model.add(layers.MaxPooling2D((2, 2)))
      if(batch_norm):
        model.add(layers.BatchNormalization())

  model.add(layers.Flatten())
  model.add(layers.Dense(ff_param[0], activation=ff_param[1]))
  if(dropout):
    model.add(layers.Dropout(.2))
  model.add(layers.Dense(nr_classes))
  
  return model

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'validation_loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric


parameters_dict = {
    'num_filter1': {'values': [16,32,64,128,256]},
    'num_filter2': {'values': [16,32,64,128,256]},
    'num_filter3': {'values': [16,32,64,128,256]},
    'num_filter4': {'values': [16,32,64,128,256]},
    'num_filter5': {'values': [16,32,64,128,256]},
    'num_fc' : {'values' : [128,256,512]},
    'epochs': {'value': 25},
    'data_aug' : {'values' : [True, False]},
    'drop_out' : {'values' : [True, False]},
    'batch_norm' : {'values' : [True, False]},
    }

sweep_config['parameters'] = parameters_dict

#sweep_id = wandb.sweep(sweep_config)
import pprint

pprint.pprint(sweep_config)

In [ ]:
!pip install wandb
!wandb login

In [ ]:
import wandb
sweep_id = wandb.sweep(sweep_config, project="CNN on inaturalist dataset")

In [ ]:
from wandb.keras import WandbCallback
def train(config=None):
    with wandb.init(config=config) as run:

        config = wandb.config
        
        cnn_params = [[config.num_filter1,3,'relu'],[config.num_filter2,3,'relu'],[config.num_filter3,3,'relu'],[config.num_filter4,3,'relu'],[config.num_filter5,3,'relu']]
        ff_params = [config.num_fc,'relu']

        input_size = (224,224,3)
        num_classes = 10
        data_aug = config.data_aug
        dropout = config.drop_out
        batch_norm = config.batch_norm

        model = model_creation(cnn_params,ff_params,input_size, num_classes,data_aug,dropout,batch_norm)
        
        model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

        #history = model.fit(train_ds,validation_data=val_ds,epochs=config.epochs)
        model.fit(train_ds,validation_data=val_ds,epochs=config.epochs,callbacks=[WandbCallback()])
count = 10 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)
#wandb.agent(sweep_id, function=train, count=2)